In [1]:
import math
import numpy as np
import pandas as pd
from fun_utils import *
from scipy.optimize import minimize, least_squares
from scipy.optimize import fsolve
from scipy.integrate import quad
import matplotlib.pyplot as plt

In [2]:
## scenario switch
case=3;  # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0; # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [3]:
## parameter values
alpha = 0.85;           # labor share parameter in manufacturing
theta = 4;              # scopevec for comparative advantage
sigma = 1;      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1;  # elasticity of demand for each individual manufactured good j at Foreign
epsilonD = alpha + (1 - alpha) * sigma;  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar;  #Foreign's elasticity of demand for embodied energy
# beta = 2.274853;
# gamma= 0.784877595;
beta=1.892412;
gamma=0.807998928;
epsilonS = 0.5;  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5;  #Foreign's energy supply elasticity: betastar/(1 - betastar)

In [11]:
epsilonD

1.0

In [4]:
ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)

In [5]:
df = pd.read_csv("../output/BaselineCarbon_2015.csv", index_col=['regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF'])
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF'])
#df=df.drop([1,2,4,5,6,7])  
df

,region_scenario,Qe,Qestar,Qeworld,CeHH,CeHF,CeFH,CeFF,Ce,Cestar,Ge,Gestar,Ceworld,Geworld,jxbar,jmbar
regionbase,,,,,,,,,,,,,,,,
US as Home,1,4.480045,27.795946,32.275990,4.598403,1.196111,0.421601,26.059874,5.794514,26.481474,5.020004,27.255985,32.275990,32.275990,0.015921,0.793579
EU28 as Home,2,0.935781,31.340202,32.275986,2.950581,1.013552,0.507676,27.804173,3.964133,28.311848,3.458257,28.817724,32.275982,32.275982,0.017932,0.744319
OECD37 as Home,3,8.625495,23.650503,32.276001,11.293668,2.487537,0.910579,17.584217,13.781205,18.494795,12.204247,20.071754,32.276001,32.276001,0.049234,0.819498
World as Home,4,32.175999,0.100000,32.276001,32.146000,0.030000,0.030000,0.070000,32.175999,0.100000,32.175999,0.100000,32.276001,32.276001,0.300000,0.999068
China as Home,5,7.522744,24.753246,32.275990,7.345464,0.632472,1.935382,22.362669,7.977936,24.298052,9.280846,22.995142,32.275990,32.275990,0.079652,0.920722
OECD and China as Home,6,16.148245,16.127758,32.276001,20.103180,1.655961,1.381915,9.134943,21.759140,10.516858,21.485096,10.790904,32.276001,32.276001,0.131400,0.923896
US and EU as Home,7,5.415825,26.860159,32.275986,7.728876,2.029770,0.749387,21.767950,9.758646,22.517336,8.478263,23.797718,32.275982,32.275982,0.033280,0.792003


In [6]:
tax_scenario= pd.DataFrame({'tax_sce': 'Unilateral', 'Base':0},index=[1])

In [7]:
def iterate_varphi(tax_scenario, df, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
    #varphilist = np.arange (1.6,2.5,0.1) # marginal damages
    varphilist = np.append((np.append(np.arange(0,35,0.1), (np.arange(35,200,2)))), np.arange(200,1000,5)) # marginal damages

    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    #varphilist = [2]
    output=[]
    
    # initial guess for optimizers, for subsequent optimizations the initial guess will be the previous optimal value
    prevtb = 0
    prevpe = 1
    
    for varphi in varphilist:
        # nested minimizer with sum of absolute value difference as objective
        #print(df)        
        tax_df = df.apply(assign_eq, axis = 1, args = (prevtb, prevpe, varphi, ParaList, tax_scenario))
        prevtb = tax_df[['tb', 'region']]
        prevpe = tax_df[['pe', 'region']]
        
        appended_df = pd.merge(df, tax_df, on=['regionbase'])

        ## back out other optimal values
        output_df = appended_df.apply(compute_all, axis=1, raw=False, result_type=None, args=(varphi, tax_scenario, ParaList))
        output_df = pd.merge(tax_df, output_df, on=['regionbase'])
        output.append(output_df)
        #print(varphi)

    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region','varphi'])

    print(tax_scenario['tax_sce'])
    return output


In [8]:
def objective(p, varphi, tax_scenario, ParaList, df):
    pe = p[0]
    tb_mat = p[1:]
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList

    #solve for equilibrium
    if tax_scenario['tax_sce']!='Unilateral':
        print("shouldn't be here")
        return 0
        
     ## optimal values
    # jxbar_hat =   (1 - df['jxbar']) ** (-1) / (((1 - df['jxbar']) ** (-1) - 1) + (1 + (1 - alpha) * tb_mat[0]/pe) ** (-theta) * (1 + tb_mat[0]/pe) ** ((1 - alpha) * theta));
    jxbar_hat = pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) / ( df['jxbar'] * pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * (pe + (1-alpha) * tb_mat[0])**(-theta));
    j0_hat = (pe+tb_mat[0])**(-(1-alpha)*theta) / (df['jxbar'] * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * pe**(-(1-alpha)*theta))
    jmbar_hat = 1
    
    if tax_scenario['tax_sce']=='Unilateral':
        te=varphi;

    
    jxbar_prime = jxbar_hat * df['jxbar'];
    jmbar_prime = jmbar_hat * df['jmbar'];
    
    j0_prime = j0_hat * df['jxbar'];

    def tempFunction(i, theta, sigmastar):
        return (i ** ((1 + theta) / theta - 1) * (1 - i) ** ((theta - sigmastar) / theta - 1)) 
    
    incomp_beta1 = quad(tempFunction,0,j0_prime, args=(theta, sigmastar))
    incomp_beta2 = quad(tempFunction,0,jxbar_prime, args=(theta, sigmastar))
    Bfunvec1_prime = incomp_beta1[0]
    Bfunvec2_prime = incomp_beta2[0]
    if (incomp_beta1[1] > 0.01 or incomp_beta2[1] > 0.01):
        print('integral convergence failed')

    #if te is too large, Home stop producing
    petbte = pe + tb_mat[0] - te;
    z = pe + tb_mat[0] >= te;
    petbte = petbte * z;

    Qe_hat = (petbte) ** epsilonS;
    Qestar_hat = pe ** epsilonSstar;
          
    if logit==1:
        epsilonS=beta*(1-gamma)/(1-gamma+gamma*petbte**beta);
        epsilonSstar=beta*(1-gamma)/(1-gamma+gamma*pe**beta);
        Qe_hat = (petbte)**beta/(1-gamma+gamma*(petbte)**beta);
        Qestar_hat = pe**beta/(1-gamma+gamma*pe**beta);
 
    Qe_prime = df['Qe'] * Qe_hat;
    Qestar_prime = df['Qestar'] * Qestar_hat;
     
    CeHH_hat = (pe + tb_mat[0]) ** (-epsilonD) * jmbar_hat ** (1 + (1 - sigma)/theta);
    CeHH_prime = df['CeHH'] * CeHH_hat;
       
    
    # CeFH_hat = (1 + (1 - sigmastar)/theta) * pe ** (-(1 - alpha) * sigmastar) * (pe + tb_mat[0]) ** (-alpha) * Bfunvec_prime/(df['jxbar'] ** (1 +1/theta)) * (1 - df['jxbar']) ** (sigmastar/theta);
    CeFH1_hat = (pe +tb_mat[0])**(-epsilonDstar) * j0_hat**(1 + (1 - sigmastar)/theta);
    CeFH2_hat = (1 + (1 - sigmastar)/theta) * ((1-df['jxbar'])/df['jxbar'])**(sigmastar/theta) * pe**(-epsilonDstar) * (1 + tb_mat[0]/pe)**(-alpha) * (Bfunvec2_prime - Bfunvec1_prime)/df['jxbar']**(1+(1-sigmastar)/theta);
    CeFH1_prime = df['CeFH'] * CeFH1_hat;
    CeFH2_prime = df['CeFH'] * CeFH2_hat;
    CeFH_hat = CeFH1_hat + CeFH2_hat;
    
    if np.isnan(CeFH_hat)==True:
        CeFH_hat=0
    CeFH_prime =df['CeFH'] * CeFH_hat;
    
    
    CeHF_hat = (pe + tb_mat[0]) ** (-epsilonD);
    CeHF_prime = df['CeHF'] * CeHF_hat;
    
    
    CeFF_prime = df['CeFF'] * ((1 - jxbar_prime)/(1-df['jxbar'])) ** (1 + (1 - sigmastar)/theta) * pe ** (-epsilonDstar)
    
    ##
    VgHH = df['CeHH']/(1 - alpha);
    VgFF = df['CeFF']/(1 - alpha);
    
    VgFH = df['CeFH'] /(1 - alpha);
    # VgFH_prime = VgFH * pe ** ((1 - sigmastar) * (1 - alpha)) * (1 - (1 - jxbar_prime) ** (1 + (1 - sigmastar)/theta))/ (df['jxbar'] * (1 - df['jxbar']) ** ( (1-sigmastar)/theta));
    VgFH1_hat = (pe + tb_mat[0]) * CeFH1_hat;
    VgFH2_hat = pe**(1 - epsilonDstar) * ((1-j0_prime)**(1+(1-sigmastar)/theta) - (1-jxbar_prime)**(1+(1-sigmastar)/theta))/ (df['jxbar']  * (1 - df['jxbar'] )**( (1-sigmastar)/theta));
    VgFH1_prime = VgFH * VgFH1_hat;
    VgFH2_prime = VgFH * VgFH2_hat;
    
    
    diff = Qe_prime + Qestar_prime - (CeHH_prime + CeFH_prime + CeHF_prime + CeFF_prime);
    pai_g = (pe + tb_mat[0]) * CeFH2_prime / (1 - alpha) - VgFH2_prime
    
    numerator = varphi * epsilonSstar * Qestar_prime - sigmastar * (1-alpha) * pai_g
    denominator = epsilonSstar * Qestar_prime + epsilonDstar * CeFF_prime
    diff1 = tb_mat[0] * denominator - numerator
        
    return abs(diff) + abs(diff1)

def assign_eq(df, prevtb, prevpe, varphi, ParaList, tax_scenario):
    region = df['region_scenario']
    if type(prevtb) != int:
        #print(prevtb)
        #print(prevtb[prevtb['region'] == region])
        prevtb = prevtb[prevtb['region'] == region]['tb']
        prevpe = prevpe[prevpe['region'] == region]['pe']
        
    prevpe, prevtb = find_opt(prevpe, prevtb, varphi, tax_scenario, ParaList, df, region)
        
    return pd.Series({'pe': prevpe, 'tb': prevtb, 'region': region})

# program to grid search optimal starting point to find equilibrium tax and price, if no statement printed then equilibrium price is found
# returns the tuple of equilibrium price and tax, if no equilibrium found, return the previous tb and pe
def find_opt(prevpe, prevtb, varphi, tax_scenario, ParaList, df, region):
    retpe = -1
    rettb = -1
    threshold = 0.01
    # grid search for pe, since the objective is unstable in pe space
    for pe in np.append(prevpe, np.arange(0,1.1,0.1)):
        res1 = minimize(objective, (pe, prevtb), args = (varphi, tax_scenario, ParaList, df), bounds = [(0,np.inf), (0, np.inf)], method = 'Nelder-Mead', options = {'maxfev': 100000})
        diff = verify([res1.x[1]], res1.x[0], varphi, tax_scenario, ParaList, df)
        if (diff[0] < threshold and diff[1] < threshold):
            retpe = res1.x[0]
            rettb = res1.x[1]
            #print(diff)
            break
    if (retpe == -1):
        print('no result found, returning placeholder (previous value)', varphi, region)
        retpe = 0.1
        rettb = 0.1
    return (retpe, rettb)

In [9]:
output_all=tax_scenario.apply(iterate_varphi, axis=1, args=(df, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))

1.0 38.63009133333333
1.0 26.42755333333333
1.0 91.87469999999999
1.0 214.50665999999995
1.0 53.18624199999999
1.0 145.0609333333333
1.0 65.05763999999999


C:\Users\Alex\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_minimize.py:496: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x0 = np.asarray(x0)


1.033493340508131 38.63009133333333
1.0333663233333372 26.42755333333333
1.033647423658919 91.87469999999999
1.03435982035473 214.50665999999995
1.0335956889343167 53.18624199999999
1.0339195832264445 145.0609333333333
1.033523153144372 65.05763999999999


C:\Users\Alex\AppData\Local\Temp/ipykernel_4652/3447501845.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  jxbar_hat = pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) / ( df['jxbar'] * pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * (pe + (1-alpha) * tb_mat[0])**(-theta));
C:\Users\Alex\AppData\Local\Temp/ipykernel_4652/3447501845.py:13: RuntimeWarning: invalid value encountered in double_scalars
  jxbar_hat = pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) / ( df['jxbar'] * pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * (pe + (1-alpha) * tb_mat[0])**(-theta));
C:\Users\Alex\AppData\Local\Temp/ipykernel_4652/3447501845.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  j0_hat = (pe+tb_mat[0])**(-(1-alpha)*theta) / (df['jxbar'] * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * pe**(-(1-alpha)*theta))
C:\Users\Alex\AppData\Local\Temp/ipykernel_4652/3447501845.py:59: Ru

no result found, returning placeholder (previous value) 0.2 4.0
1.067312813061874 38.63009133333333
1.0668063933999739 26.42755333333333
1.0679405418153656 91.87469999999999
0.2 214.50665999999995
1.0677561616742883 53.18624199999999
1.0690703795988699 145.0609333333333
1.0674660440337158 65.05763999999999
1.1014943699125972 38.63009133333333
1.1003245487024347 26.42755333333333
1.1029327501828483 91.87469999999999
1.1106270684075765 214.50665999999995
1.1025105594250963 53.18624199999999
1.1055331174499505 145.0609333333333
1.1018332314880446 65.05763999999999
1.136062802129926 38.63009133333333
1.1339249325119305 26.42755333333333
1.1386703926819426 91.87469999999999
1.1526008498062401 214.50665999999995
1.137911590193688 53.18624199999999
1.1433776166745497 145.0609333333333
1.1366834421983545 65.05763999999999
1.171043732637541 38.63009133333333
1.1676142153904738 26.42755333333333
1.1752231920738627 91.87469999999999
1.197337681466501 214.50665999999995
1.174010899433449 53.186241

KeyboardInterrupt: 

In [10]:
output_list=[]
for i in range(1,len(tax_scenario)+1):
    output_list.append(output_all.loc[i])
outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
outcomes.reset_index(level=0, inplace=True)

In [11]:
outcomes

,tax_sce,regionbase,pe,tb,region,varphi,jxbar_prime,jmbar_prime,j0_prime,Qe_prime,...,leakage2,leakage3,chg_extraction,chg_production,chg_consumption,chg_Qeworld,pai_g,subsidy_ratio,welfare,welfare_noexternality
0,Unilateral,US as Home,1.000000,0.000000,1.0,0.0,0.015921,0.793579,0.015921,4.480045,...,-0.000000,-1.287500,0.000000,0.000000,0.000001,8.000000e-07,0.000000,0.000000,-0.000003,-0.000003
7,Unilateral,US as Home,0.998753,0.034740,1.0,0.1,0.015925,0.793579,0.015602,4.328505,...,-0.037656,0.111829,-0.017332,-0.006359,0.018885,-1.688705e-01,-0.094139,0.005190,0.021790,-0.021925
14,Unilateral,US as Home,0.997897,0.069416,1.0,0.2,0.015939,0.793579,0.015301,4.172249,...,-0.062304,0.084225,-0.029245,-0.020999,0.028387,-3.370401e-01,-0.189194,0.010327,0.087318,-0.087179
21,Unilateral,US as Home,0.997417,0.104077,1.0,0.3,0.015961,0.793579,0.015014,4.010815,...,-0.086713,0.056840,-0.035918,-0.043803,0.028713,-5.051476e-01,-0.285268,0.015411,0.196201,-0.196095
28,Unilateral,US as Home,0.997303,0.138760,1.0,0.4,0.015991,0.793579,0.014741,3.843614,...,-0.110742,0.029776,-0.037512,-0.074634,0.020067,-6.739415e-01,-0.382428,0.020444,0.349017,-0.348824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4122,Unilateral,US and EU as Home,0.001638,344.791743,7.0,975.0,1.000000,0.792003,0.000022,0.000000,...,-0.762831,-0.688019,-25.773033,-23.791831,-21.458513,-3.118886e+01,-1717.561319,0.999968,43907.030149,-2834.810669
4129,Unilateral,US and EU as Home,0.001627,346.524106,7.0,980.0,1.000000,0.792003,0.000022,0.000000,...,-0.762744,-0.688050,-25.776627,-23.791861,-21.461966,-3.119245e+01,-1726.215995,0.999969,44146.745649,-2840.211224
4136,Unilateral,US and EU as Home,0.001617,348.256275,7.0,985.0,1.000000,0.792003,0.000022,0.000000,...,-0.762658,-0.688081,-25.780192,-23.791890,-21.465390,-3.119602e+01,-1734.869706,0.999969,44386.488855,-2845.594060
4143,Unilateral,US and EU as Home,0.001606,349.988425,7.0,990.0,1.000000,0.792003,0.000022,0.000000,...,-0.762572,-0.688112,-25.783726,-23.791918,-21.468786,-3.119955e+01,-1743.523320,0.999969,44626.259092,-2850.959333


In [12]:
outcomes.to_csv('../output/convergent.csv'.format(case), header=True) 